In [ ]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from datetime import datetime


import statsmodels.api as sm

In [ ]:
def get_model(nodes_in_hidden_layers, dropout, nr_of_inputs):
    model = Sequential()
    for i in range(len(nodes_in_hidden_layers)):
        if i == 0:
            model.add(Dense(nodes_in_hidden_layers[i], activation='swish', input_shape=(nr_of_inputs,)))  # Input and hidden layer
        else:
            if dropout != 0:
                model.add(Dropout(dropout))
            model.add(Dense(nodes_in_hidden_layers[i], activation='swish'))  # Additional hidden layers
    model.add(Dense(1, activation="linear"))  # Output layer
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mean_squared_error", metrics=("mean_squared_error"))
    return model

In [ ]:
def get_base_features(data):
    station = np.eye(19)[data["Base Station"]]
    year = np.eye(4)[data["Year"] - 2015]
    month = np.eye(12)[data["Month"] - 1]
    day = np.eye(31)[data["Day"] - 1]
    week = np.eye(53)[data["Week"] - 1]
    weekday = np.eye(7)[data["Weekday"] - 1]
    hour = np.eye(24)[data["Hour"]]

    x = np.concatenate([
        station,
        year, 
        month, 
        day, 
        week, 
        weekday,
        hour], axis=1)
    return x

In [ ]:
def get_features(data):
    station = np.eye(19)[data["Base Station"]]
    year = np.eye(4)[data["Year"] - 2015]
    month = np.eye(12)[data["Month"] - 1]
    day = np.eye(31)[data["Day"] - 1]
    week = np.eye(53)[data["Week"] - 1]
    weekday = np.eye(7)[data["Weekday"] - 1]
    hour = np.eye(24)[data["Hour"]]
    daytime = np.zeros((len(data), 6))

    # Determine season based on month (assuming 4 seasons: 0-Winter, 1-Spring, 2-Summer, 3-Autumn)
    season = np.zeros((len(data), 4))
    season[:, 0] = np.logical_or(month[:, 0] == 11, month[:, 0] <= 2)  # Winter
    season[:, 1] = np.logical_and(month[:, 0] >= 3, month[:, 0] <= 5)  # Spring
    season[:, 2] = np.logical_and(month[:, 0] >= 6, month[:, 0] <= 8)  # Summer
    season[:, 3] = np.logical_and(month[:, 0] >= 9, month[:, 0] <= 10)  # Autumn
    
    # Determine if it's a weekend (1-Weekend, 0-Not a weekend)
    weekend = np.logical_or(weekday[:, 5] == 1, weekday[:, 6] == 1)
    weekend = np.expand_dims(weekend.astype(int), axis=1)
    
    # Loop over the four-hour periods and sum the one hot encoded values of each hour in the period
    for i in range(6):
        daytime[:, i] = np.sum(hour[:, i*4:(i+1)*4], axis=1)

    x = np.concatenate([
        station,
        year, 
        month, 
        day, 
        week, 
        weekday,
        hour,
        season,
        weekend,
        #daytime
    ], axis=1)
    
    return x

In [ ]:
def get_original_features(data):
    def get_daytime(hour, period_hours):
        periods = int(24/period_hours)
        for i in range(periods):
            start_hour = i
            end_hour = (i+1) * period_hours
            if start_hour <= hour < end_hour:
                return i
        return 6

    # Map the Hour column to the Daytime column using the map_hour_to_daytime function
    data["Daytime"] = data["Hour"].apply(lambda x: get_daytime(x, 4))

    return data[["Base Station", "Year", "Month", "Day", "Week", "Weekday", "Hour"]].values

In [ ]:
def revert_base_features(data):
    base_station = np.argmax(data[:, :19], axis=1) 
    year = np.argmax(data[:, 19:23], axis=1) + 2015
    month = np.argmax(data[:, 23:35], axis=1) + 1
    day = np.argmax(data[:, 35:66], axis=1) + 1
    week = np.argmax(data[:, 66:119], axis=1) + 1
    weekday = np.argmax(data[:, 119:126], axis=1) + 1
    hour = np.argmax(data[:, 126:], axis=1)

    return list(zip(base_station, year, month, day, week, weekday, hour))

In [ ]:
def revert_features(data):
    base_station = np.argmax(data[:, :19], axis=1) 
    year = np.argmax(data[:, 19:23], axis=1) + 2015
    month = np.argmax(data[:, 23:35], axis=1) + 1
    day = np.argmax(data[:, 35:66], axis=1) + 1
    week = np.argmax(data[:, 66:119], axis=1) + 1
    weekday = np.argmax(data[:, 119:126], axis=1) + 1
    hour = np.argmax(data[:, 126:150], axis=1)
    season = np.argmax(data[:, 150:154], axis=1)
    weekend = np.argmax(data[:, 154:155], axis=1)

    return list(zip(base_station, year, month, day, week, weekday, hour, season, weekend))

In [ ]:
def train_test_split_date(data, start_date, end_date):
    datetime_columns = ['Year', 'Month', 'Day', 'Hour']
    datetime_data = pd.to_datetime(data[datetime_columns])
    
    mask = (datetime_data.dt.floor('h').between(start_date, end_date, inclusive=True))

    data_train = data[~mask]    
    data_test = data[mask]

    return data_train, data_test

In [ ]:
def preprocess(data, s):
    data_station = data
    if s != None:
        data_station = data[data["Base Station"] == s]
    x = get_features(data_station)
    y = data_station["Incidents"].to_numpy()
    return x, y

In [ ]:
DISTRIBUTION_FILE = "data/incident_distribution/station.csv"

data = pd.read_csv(DISTRIBUTION_FILE, encoding='utf-8', escapechar='\\', parse_dates=True)

In [ ]:
start_date = datetime(2017, 8, 7, 0, 0, 0)
end_date = datetime(2017, 8, 13, 23, 59, 59)
data_train, data_test = train_test_split_date(data, start_date, end_date)

""" start_date = datetime(2017, 8, 6)
end_date = datetime(2017, 8, 22)
_, data_validation = train_test_split_date(data, start_date, end_date)
X_validation, Y_validation = preprocess(data_test, None) """

In [ ]:
start_date = datetime(2018, 8, 1, 0, 0, 0)
end_date = datetime(2018, 8, 31, 0, 0, 0)
data_train, data_val = train_test_split_date(data_train, start_date, end_date)

X_train, Y_train = preprocess(data_train, None)
X_test, Y_test = preprocess(data_test, None)
X_val, Y_val = preprocess(data_val, None)

#X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

layers = [64, 32]
model = get_model(layers, 0.4, len(X_train[0]))
es = EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=10)
model.fit(X_train, Y_train, epochs=30, validation_data=(X_val, Y_val), verbose=1, callbacks=[es])

In [ ]:
start_date = datetime(2018, 8, 1, 0, 0, 0)
end_date = datetime(2018, 8, 31, 0, 0, 0)
data_train, data_val = train_test_split_date(data_train, start_date, end_date)

X_train, Y_train = preprocess(data_train, None)
X_test, Y_test = preprocess(data_test, None)
X_val, Y_val = preprocess(data_val, None)

In [ ]:
ce = []
layerss = [[64]]
for layers in layerss:
    for _ in range(5):
        model = get_model(layers, 0.4, len(X_train[0]))
        es = EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=5)
        history = model.fit(X_train, Y_train, epochs=30, validation_data=(X_val, Y_val), verbose=1, callbacks=[es])
        ce.append(min(history.history['val_mean_squared_error']))
    avg_ce = sum(ce) / len(ce)
    print(layers, avg_ce)

In [ ]:
poisson_model = sm.GLM(Y_train, X_train, family=sm.families.Poisson())
poisson_results = poisson_model.fit()
val_pred_poisson = poisson_results.predict(X_val)
mse_poisson = mean_squared_error(Y_val, val_pred_poisson)
print("Mean squared error (Poisson):", mse_poisson)

nb_model = sm.GLM(Y_train, X_train, family=sm.families.NegativeBinomial())
nb_results = nb_model.fit()
val_pred_nb = nb_results.predict(X_val)
mse_nb = mean_squared_error(Y_val, val_pred_nb)
print("Mean squared error (NB):", mse_nb)

""" zinb_model = sm.ZeroInflatedNegativeBinomialP(endog=Y_train, exog=X_train, exog_infl=np.log(X_train + 1e-8))
zinb_results = zinb_model.fit()
val_pred_zinb = zinb_results.predict(X_val, exog_infl=np.log(X_val + 1e-8))
mse_zinb = mean_squared_error(Y_val, val_pred_zinb)
print("Mean squared error (ZINB):", mse_zinb)

zip_model = sm.ZeroInflatedPoisson(endog=Y_train, exog=X_train, exog_infl=np.log(X_train + 1e-8))
zip_results = zip_model.fit()
val_pred_zip = zip_results.predict(X_val, exog_infl=np.log(X_val + 1e-8))
mse_zip = mean_squared_error(Y_val, val_pred_zip)
print("Mean squared error (ZIP):", mse_zip) """

In [ ]:
def cross_validation(x_train, y_train):
    kf = KFold(n_splits=5)
    ce = []
    for train_index, validation_index in kf.split(x_train):
        model = get_model([128, 32, 8], 0.4, len(x_train[0]))
        es = EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=5)
        history = model.fit(x_train[train_index], y_train[train_index], validation_data=(x_train[validation_index], y_train[validation_index]), epochs=30, verbose=1, callbacks=[es])
        ce.append(min(history.history['val_mean_squared_error']))
    avg_ce = sum(ce) / len(ce)
    return avg_ce

In [ ]:
X_train, Y_train = preprocess(data_train, None)
X_test, Y_test = preprocess(data_test, None)

avg = cross_validation(X_train, Y_train)
print(avg)

In [ ]:
predictions = []
Y_test = []
X_test = []

for s in range(0, 19):
    X_train, Y_train = preprocess(data_train, s)
    X_test_s, Y_test_s = preprocess(data_test, s)

    layers = [128, 64, 16]
    model = get_model(layers, 0.4, len(X_train[0]))
    es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=1)
    model.fit(X_train, Y_train, epochs=20, verbose=1, callbacks=[es])

    """# Fit the Poisson regression model
    model = sm.GLM(Y_train, X_train, family=sm.families.Poisson())
    model = sm.GLM(Y_train, X_train, family=sm.families.NegativeBinomial())
    model = sm.ZeroInflatedGeneralizedPoisson(Y_train, X_train, exog_infl=X_train, inflation='logit')
    model = model.fit() """

    predictions_station = model.predict(X_test_s, verbose=0)
    print(mean_squared_error(Y_test_s, predictions_station))

    predictions.extend(predictions_station)
    X_test.extend(X_test_s)
    Y_test.extend(Y_test_s)

predictions = np.array(predictions)
Y_test = np.array(Y_test)
X_test = np.array(X_test)

In [ ]:
# Fit the Poisson regression model
model = sm.GLM(Y_train, X_train, family=sm.families.Poisson())
model = model.fit()

# Print the model summary
print(model.summary())

In [ ]:
predictions = model.predict(X_test)

In [ ]:
mse = mean_squared_error(Y_test, predictions)
print("Mean squared error: ", mse)

X_test_reverted = revert_features(X_test)

print("Base Station", "Year", "Month", "Day", "Week", "Weekday", "Hour", "Season", "Weekend", "Daytime")
for i in range(len(predictions)):
    print(predictions[i], np.round(predictions[i], 2), Y_test[i], X_test_reverted[i])

In [ ]:
PREDICTIONS_FILE = "data/incident_distribution/station_predictions_3.json"

def save_predictions(predictions, X_test_reverted):
    d = {}
    for station in range(0, 19):
        d[station] = {}
        for day in range(6, 14):
            d[station][day] = {}
            for hour in range(0, 24):
                d[station][day][hour] = {}
    for prediction, x in zip(predictions, X_test_reverted):

        station = x[0]
        day = x[3]
        hour = x[6]
        if day == 14:
            continue
    
        d[station][day][hour] = round(float(prediction), 3)
    print("Saving predictions to file...")
    with open(PREDICTIONS_FILE, 'w') as f:
        json.dump(d, f, indent=2)


save_predictions(predictions, X_test_reverted)

In [ ]:
PREDICTIONS_FILE = "data/incident_distribution/station_predictions_3.json"

def save_predictions(predictions, X_test_reverted):
    d = {}
    with open(PREDICTIONS_FILE, 'r') as f:
        d = json.load(f)
    for station in range(0, 19):
        for day in range(14, 22):
            d[str(station)][str(day)] = {}
            for hour in range(0, 24):
                d[str(station)][str(day)][str(hour)] = {}
    for prediction, x in zip(predictions, X_test_reverted):

        station = x[0]
        day = x[3]
        hour = x[6]
    
        d[str(station)][str(day)][str(hour)] = round(float(prediction), 3)
    print("Saving predictions to file...")
    with open(PREDICTIONS_FILE, 'w') as f:
        json.dump(d, f, indent=2)


save_predictions(predictions, X_test_reverted)